In [ ]:
import math
import cv2
import os
import numpy as np
import time
import random
import torch
import re
import torch.nn as nn
import torchvision.models as models
import torch.autograd as autograd
import torch.optim as optim
from torchvision import datasets ,models,transforms
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from PIL import Image, ImageEnhance
import torchvision.transforms.functional as TF
from torch.utils.tensorboard import SummaryWriter

SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [ ]:
class MyRotationTransform:
    """Rotate by one of the given angles."""

    def __init__(self, angles):
        self.angles = angles

    def __call__(self, x):
        angle = random.choice(self.angles)
        return TF.rotate(x, angle)

class mydataset(Dataset):
    def __init__(self, path,train=False):
        self.path=path
        self.namelist=os.listdir(self.path)
        self.labellist=[]
        if train:
            self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.RandomCrop(224),
            MyRotationTransform(angles=[0,180]),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            ])
        else:
            self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            ])

        self.patterns=['grid','hony','wave','rect']
        for i in range(len(self.namelist)):
            filename=self.namelist[i]
            try:
                pattern, has_exterior, defects = re.match(
                    "(grid|hony|wave|rect)\d+_\d+_(\d)_\d(\d\d\d).png", filename).groups()
            except Exception as e:
                print(f'failed to load {filename}', e)
                continue
            pattern = self.patterns.index(pattern)
            defects = np.array(list(defects), dtype=float)
            if int(has_exterior) == 1:
                has_exterior=1
            else:
                has_exterior=0

            self.namelist[i]=[filename,{'defect':defects, 'pattern':pattern, 'has_exterior':has_exterior}]

    def __getitem__(self, item):
        name = self.namelist[item][0]
        label = self.namelist[item][1]
        image = Image.open(self.path+name).convert('F')
        #image = self.transform(image)
            
        return name, image, label
        
    def __len__(self):
        return len(self.namelist)

In [ ]:
train_data = mydataset('/content/drive/MyDrive/Colab Notebooks/dataset/training/')
val_data = mydataset('/content/drive/MyDrive/Colab Notebooks/dataset/validation/')

In [ ]:
import numpy as np
from skimage import feature as ft
from tqdm.autonotebook import tqdm


NUM_FEATURES = 12800
NUM_DEFECTS = 3

print(len(train_data))


def collect_dataset(dataset):
  features = np.empty((len(dataset), NUM_FEATURES))
  labels = np.empty((len(dataset), NUM_DEFECTS))

  for (i, (_, img, labels_dict)) in tqdm(enumerate(dataset), total=len(dataset)):
    features[i] = ft.hog(img,
                   orientations = 8, 
                   pixels_per_cell = (16, 16), 
                   cells_per_block = (8, 8), 
                   block_norm = 'L2-Hys', 
                   transform_sqrt = True)
    labels[i] = labels_dict['defect']

  return features, labels

(train_data, train_labels) = collect_dataset(train_data)
(val_data, val_labels) = collect_dataset(val_data)

5376


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

models = [make_pipeline(StandardScaler(), SVC(gamma='auto')) for _ in range(NUM_DEFECTS)]

for i in range(NUM_DEFECTS):
  print(f'fitting defect {i}')
  models[i].fit(train_data, train_labels[:, i])

fitting defect 0
fitting defect 1
fitting defect 2


In [ ]:
predicted_defects = np.empty((len(val_data), NUM_DEFECTS))

for i, model  in enumerate(models):
  print(f'accuracy on defect {i}: ', end='')
  score = model.score(val_data, val_labels[:, i])
  predicted_defects[:, i] = model.predict(val_data)
  print(f'{score * 100:.1f}%')

tp = ((predicted_defects == True) & (val_labels == True)).sum()
fp = ((predicted_defects == True) & (val_labels == False)).sum()
fn = ((predicted_defects == False) & (val_labels == True)).sum()
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)
print(f'precision: {precision:.3f}')
print(f'recall: {recall:.3f}')
print(f'f1 score: {f1_score:.3f}')

accuracy on defect 0: 78.6%
accuracy on defect 1: 68.6%
accuracy on defect 2: 76.2%
precision: 0.724
recall: 0.553
f1 score: 0.627


In [ ]:
test_dataset = mydataset('/content/drive/MyDrive/Colab Notebooks/dataset/testing/')
(test_data, test_labels) = collect_dataset(test_dataset)
predicted_defects = np.empty((len(test_data), NUM_DEFECTS))

for i, model  in enumerate(models):
  print(f'accuracy on defect {i}: ', end='')
  score = model.score(test_data, test_labels[:, i])
  predicted_defects[:, i] = model.predict(test_data)
  print(f'{score * 100:.1f}%')

tp = ((predicted_defects == True) & (test_labels == True)).sum()
fp = ((predicted_defects == True) & (test_labels == False)).sum()
fn = ((predicted_defects == False) & (test_labels == True)).sum()
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)
print(f'precision: {precision:.3f}')
print(f'recall: {recall:.3f}')
print(f'f1 score: {f1_score:.3f}')


accuracy on defect 0: 81.0%
accuracy on defect 1: 70.1%
accuracy on defect 2: 76.6%
precision: 0.752
recall: 0.566
f1 score: 0.646
